# Методы компрессии нейронных сетей

## Лекция №3 - Кластеризация весов моделей

*   Использование кластеризации для сокращения размера моделей.
*   Алгоритмы кластеризации и их применение к моделям машинного обучения.
*   Практика - Пример с кластеризацией весов

## ДЗ №3:
Применение данного подхода к своим моделям и замер производительности

## Домашняя работа
В качестве модели выбрана YOLOv8m-cls

Проверяются следующие метрики производительности:

*   число параметров модели;
*   вес файла модели;
*   целевая метрика.

In [1]:
!pip install -q tensorflow-model-optimization
!pip install model_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 4.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10, mnist
import tensorflow_model_optimization as tfmot
from model_profiler import model_profiler

batch_size = 128

## Загрузка датасета

In [3]:
tf.random.set_seed(1)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train / 255
x_test = x_test / 255

170498071/170498071 [==============================] - 2s 0us/step


In [4]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

## Построение модели

In [5]:
inputs = keras.Input(shape=(32, 32, 3), name="img")
x = layers.Conv2D(32, 3, activation="relu")(inputs)
x = layers.Conv2D(64, 3, activation="relu")(x)
block_1_output = layers.MaxPooling2D(3)(x)

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_1_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_2_output = layers.add([x, block_1_output])

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_2_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_3_output = layers.add([x, block_2_output])

x = layers.Conv2D(64, 3, activation="relu")(block_3_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs, name="toy_resnet")

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

### Число параметров и размер модели

In [6]:
model.weights[0][0][0]

<tf.Tensor: shape=(3, 32), dtype=float32, numpy=
array([[ 0.03174844,  0.0254668 ,  0.10575704,  0.01555993,  0.07617527,
        -0.0060412 , -0.04199091, -0.09805469, -0.07166376,  0.08314565,
        -0.1326293 , -0.05496265, -0.00557925,  0.02874334,  0.01760477,
         0.11830305, -0.12355918,  0.08025624,  0.06425397, -0.04970688,
         0.09574881,  0.05221766,  0.10128213,  0.07644835,  0.11718245,
        -0.10778049,  0.08545583,  0.0074503 , -0.10776647, -0.1170751 ,
        -0.03717163,  0.09792745],
       [ 0.04791945, -0.02236713,  0.09099188,  0.0254284 ,  0.0312704 ,
        -0.08752358,  0.13214032,  0.06848416,  0.09247656,  0.09157777,
        -0.05364866,  0.07459085, -0.06139302, -0.12345684,  0.06124169,
        -0.07472164, -0.09184933,  0.12160261,  0.03549252,  0.09716418,
        -0.08123855, -0.10297394, -0.02528951, -0.02165405,  0.07096519,
        -0.04483857, -0.10097066, -0.03261101,  0.03187075, -0.07414202,
        -0.07186714,  0.0722958 ],
     

In [7]:
model.summary()

Model: "toy_resnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img (InputLayer)            [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 30, 30, 32)           896       ['img[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)           (None, 28, 28, 64)           18496     ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 9, 9, 64)             0         ['conv2d_1[0][0]']            
 D)                                                                                      

### Занимаемая память

In [8]:
print(model_profiler(model, batch_size))

| Model Profile                    | Value   | Unit    |
|----------------------------------|---------|---------|
| Selected GPUs                    | ['0']   | GPU IDs |
| No. of FLOPs                     | 0.0006  | BFLOPs  |
| GPU Memory Requirement           | 0.0584  | GB      |
| Model Parameters                 | 0.2232  | Million |
| Memory Required by Model Weights | 0.8516  | MB      |


/usr/local/lib/python3.10/dist-packages/model_profiler/profiler.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.asarray(values).reshape(-1,1),


### Целевая метрика

In [10]:
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 7s 10ms/step - loss: 1.8178 - accuracy: 0.2874 - val_loss: 1.5169 - val_accuracy: 0.4266
Epoch 2/10
625/625 [==============================] - 5s 8ms/step - loss: 1.3922 - accuracy: 0.4783 - val_loss: 1.2242 - val_accuracy: 0.5476
Epoch 3/10
625/625 [==============================] - 5s 9ms/step - loss: 1.1699 - accuracy: 0.5767 - val_loss: 1.0615 - val_accuracy: 0.6244
Epoch 4/10
625/625 [==============================] - 6s 9ms/step - loss: 1.0172 - accuracy: 0.6373 - val_loss: 0.9672 - val_accuracy: 0.6601
Epoch 5/10
625/625 [==============================] - 5s 9ms/step - loss: 0.9054 - accuracy: 0.6770 - val_loss: 0.8453 - val_accuracy: 0.7002
Epoch 6/10
625/625 [==============================] - 6s 9ms/step - loss: 0.8355 - accuracy: 0.7068 - val_loss: 0.8382 - val_accuracy: 0.7028
Epoch 7/10
625/625 [==============================] - 5s 8ms/step - loss: 0.7513 - accuracy: 0.7377 - val_loss: 0.7494 - val_accuracy: 0.7401
Epoch

In [11]:
print(model.evaluate(x_test, y_test) )

313/313 [==============================] - 1s 4ms/step - loss: 0.7779 - accuracy: 0.7381
[0.7778986096382141, 0.738099992275238]


In [12]:
keras_file = './model.h5'
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Saving model to:  ./model.h5


<ipython-input-12-051be73342f6>:3: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


In [6]:
# model = tf.keras.models.load_model(keras_file)

## Кластеризация

In [18]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 8,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

clustered_model = cluster_weights(model, **clustering_params)
clustered_model = tfmot.clustering.keras.strip_clustering(clustered_model)

clustered_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

#### Число параметров и размер модели

In [19]:
clustered_model.weights[0][0][0]

<tf.Tensor: shape=(3, 32), dtype=float32, numpy=
array([[ 0.155607  ,  0.03271982,  0.08002635,  0.03271982,  0.155607  ,
        -0.01794856, -0.01794856, -0.1189696 , -0.06284878, -0.01794856,
        -0.16258548, -0.01794856, -0.06284878,  0.03271982,  0.03271982,
         0.155607  , -0.1189696 , -0.01794856, -0.06284878, -0.06284878,
         0.03271982, -0.01794856,  0.155607  ,  0.08002635,  0.155607  ,
        -0.16258548,  0.08002635, -0.01794856, -0.21125503, -0.16258548,
        -0.06284878,  0.155607  ],
       [ 0.155607  , -0.06284878,  0.08002635,  0.03271982,  0.155607  ,
        -0.16258548,  0.155607  ,  0.08002635,  0.155607  ,  0.03271982,
        -0.06284878,  0.08002635, -0.06284878, -0.1189696 ,  0.08002635,
        -0.06284878, -0.06284878,  0.08002635, -0.1189696 ,  0.155607  ,
        -0.06284878, -0.21125503, -0.01794856, -0.01794856,  0.08002635,
        -0.06284878, -0.16258548, -0.01794856, -0.01794856, -0.1189696 ,
        -0.06284878,  0.08002635],
     

In [20]:
clustered_model.summary()

Model: "toy_resnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img (InputLayer)            [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 30, 30, 32)           896       ['img[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)           (None, 28, 28, 64)           18496     ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 9, 9, 64)             0         ['conv2d_1[0][0]']            
 D)                                                                                      

### Занимаемая память

In [21]:
batch_size = 128
print(model_profiler(clustered_model, batch_size))

| Model Profile                    | Value   | Unit    |
|----------------------------------|---------|---------|
| Selected GPUs                    | ['0']   | GPU IDs |
| No. of FLOPs                     | 0.0006  | BFLOPs  |
| GPU Memory Requirement           | 0.0584  | GB      |
| Model Parameters                 | 0.2232  | Million |
| Memory Required by Model Weights | 0.8516  | MB      |


/usr/local/lib/python3.10/dist-packages/model_profiler/profiler.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.asarray(values).reshape(-1,1),


### Целевая метрика

In [22]:
print(clustered_model.evaluate(x_test, y_test) )

313/313 [==============================] - 2s 4ms/step - loss: 1.0937 - accuracy: 0.6011
[1.0936636924743652, 0.6011000275611877]
